In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import cv2
import csv
import math
from brian2 import *

## Paramets

In [33]:
num_classes = 2
neurons_per_class = 1
x_pixel = 8
epochs = 1


## Initialize functions

In [56]:
with open('weights.csv', 'wb') as myfile:
  x = np.random.uniform(0,1,size = [8,8])
  np.savetxt( "weights.csv", x, delimiter="," )
    
    
def read_img():

    img = cv2.imread("2.png", 0)
    img2 = cv2.imread("2.png",0)
    img = np.ndarray.flatten(img)
    return img


def read_dataset():
    df = pd.read_csv("weights.csv")
    df = df.values
    X = df
    y = df[:,0]
    return X,y

def recon_weights(W):

    for i in range(par.hid_size):
        temp = W[i]
        recon = np.reshape(temp,(par.x_pixel,par.x_pixel))*255
        cv2.imwrite("weights/weights" + str(i) + ".png",recon)


In [59]:
x = read_img()

#normalizing data
normalized = (x-np.min(x))/float(np.max(x)-np.min(x))*50 #50 Hz is the maximum firing rate

#stdp parameters
sigma = 0.3                                                                                                                                                                                                             
taupre = 5*ms
taupost = 8*ms
wmax = 1.2
wmin = -0.5
Apre = 0.4*sigma
Apost = -0.2*sigma

#characteristics of neurons
tau = 10*ms
vr = 0*volt
vt = 1.5*mV


# equations governing the dynamics of neurons
eqs = '''
dv/dt = -v/tau : volt (unless refractory)
'''
# initializing weights
S_initial = np.random.uniform(low=0,high=0.3,size=(2,64))

for i in range(1):

      for j in range(len(x)):

            #output layer neuron
            G = NeuronGroup(1, eqs, threshold='v>vt', reset='v=vr', refractory = 1.5*ms, method='linear')

            #input neurons firing according to Poisson distribution with rates determined by the intensity of the corresponding input pixel
            P = PoissonGroup(64, x[j]*Hz)

            #synapse governed by the rules of STDP
            S = Synapses(P, G,
                         '''
                         w : 1
                         dapre/dt = -apre/taupre : 1 (event-driven)
                         dapost/dt = -apost/taupost : 1 (event-driven)
                         ''',
                         on_pre='''
                         v_post += w*mV
                         apre += Apre
                         w = clip(w+apost, wmin, wmax)
                         ''',
                         on_post='''
                         apost += Apost
                         w = clip(w+apre, wmin, wmax)
                         ''')

            S.connect()
            S.w = S_initial[j]

            M = StateMonitor(G, 'v', record=True) #monitors the membrane voltage
            spikemon = SpikeMonitor(P) #records spikes from input neurons
            spikemon1 = SpikeMonitor(G) #records spikes from output neurons
            run(2*ms)

            #updating the weights
            S_initial[j] = S.w

######################################################### plots ##################################################
            # subplot(211)
            # plot(M.t/ms, M.v[0]/volt)
            # # plot(spikemon.t/ms, spikemon.i, '.k')
            # subplot(212)
            # plot(spikemon.t/ms, spikemon.i, '.k')
            # # plot(M.t/ms, M.v[0]/volt)
            # show()
###################################################################################################################

#reconstructing weights to analyse the training
recon_weights(S_initial)

IndexError: index 2 is out of bounds for axis 0 with size 2

In [1]:
def Poisson_generator(time,dt,rate, n, myseed=False):
  """
  Generates poisson trains

  Args:
    rate            : noise amplitute [Hz]
    n               : number of Poisson trains
    myseed          : random seed. int or boolean

  Returns:
    pre_spike_train : spike train matrix, ith row represents whether
                      there is a spike in ith spike train over time
                      (1 if spike, 0 otherwise)
  """

  # Retrieve simulation parameters
  dt  = 0.001
  range_t = np.arange(0, time+dt, dt)  # Vector of discretized time points [ms]
  Lt = range_t.size

  # set random seed
  if myseed:
      np.random.seed(seed=myseed)
  else:
      np.random.seed()

  # generate uniformly distributed random variables
  u_rand = np.random.rand(n, Lt)

  # generate Poisson train
  poisson_train = 1. * (u_rand < rate * (dt / 10.))

  return poisson_train

In [55]:
Poisson_generator(200,0.001,rate = 50,n=64).shape

(64, 200001)